In [6]:
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import os

BASE_DIR = os.getcwd()              # current working directory
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

os.makedirs(OUTPUT_DIR, exist_ok=True)

DATA_PATH = "C:\\Users\\HP\\Downloads\\PULSEGEN ASSESSMENT 1\\.venv\\dataset_with_random_dates.csv"
TREND_OUTPUT = os.path.join(OUTPUT_DIR, "redbus_topic_trend.csv")
DAILY_COUNT_OUTPUT = os.path.join(OUTPUT_DIR, "daily_review_count.csv")

df = pd.read_csv(DATA_PATH)
print("✅ Columns found:", list(df.columns))

df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
df = df.dropna(subset=["date"])
df = df.sort_values("date")

TOPIC_RULES = {
    "Cleanliness issue": "Feedback_Cleanliness",
    "Punctuality issue": "Feedback_Punctuality",
    "Comfort issue": "Feedback_Comfort",
    "Staff behavior issue": "Feedback_Staff",
    "Cancellation issue": "Cancelled",
    "Delay issue": "Delay_Minutes"
}

daily_topic_counts = {}
daily_review_counts = {}

unique_dates = sorted(df["date"].unique())

for day in tqdm(unique_dates, desc="📅 Processing daily batches"):
    day_df = df[df["date"] == day]
    daily_review_counts[day] = len(day_df)

    topic_counter = {}

    for topic, col in TOPIC_RULES.items():
        if col == "Delay_Minutes":
            topic_counter[topic] = int((day_df[col] > 0).sum())
        elif col == "Cancelled":
            topic_counter[topic] = int((day_df[col] == 1).sum())
        else:
            topic_counter[topic] = int((day_df[col] < 3).sum())

    daily_topic_counts[day] = topic_counter

TARGET_DATE = max(unique_dates)
START_DATE = TARGET_DATE - timedelta(days=30)

date_range = [
    START_DATE + timedelta(days=i)
    for i in range(31)
    if START_DATE + timedelta(days=i) in unique_dates
]

trend_df = pd.DataFrame(0, index=TOPIC_RULES.keys(), columns=date_range)

for day in date_range:
    for topic, count in daily_topic_counts.get(day, {}).items():
        trend_df.loc[topic, day] = count

trend_df.columns = [pd.to_datetime(d).strftime("%b %d") for d in trend_df.columns]
trend_df.index.name = "Topic"

trend_df.to_csv(TREND_OUTPUT)

daily_count_df = pd.DataFrame({
    "Date": daily_review_counts.keys(),
    "Total Reviews": daily_review_counts.values()
})
daily_count_df.to_csv(DAILY_COUNT_OUTPUT, index=False)

print("\n✅ FILES SAVED SUCCESSFULLY")
print("📁 Output folder:", OUTPUT_DIR)
print("➡ Trend file:", TREND_OUTPUT)
print("➡ Daily count file:", DAILY_COUNT_OUTPUT)


✅ Columns found: ['Trip_ID', 'Operator', 'Origin', 'Destination', 'Scheduled_Departure', 'Delay_Minutes', 'Cancelled', 'Feedback_Cleanliness', 'Feedback_Punctuality', 'Feedback_Comfort', 'Feedback_Staff', 'Tickets_Sold', 'Occupancy_Rate(%)', 'date']


📅 Processing daily batches: 100%|██████████████████████████████████████████████████| 491/491 [00:00<00:00, 542.15it/s]


✅ FILES SAVED SUCCESSFULLY
📁 Output folder: C:\Users\HP\output
➡ Trend file: C:\Users\HP\output\redbus_topic_trend.csv
➡ Daily count file: C:\Users\HP\output\daily_review_count.csv
